In [ ]:
import os
import random
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, callbacks, utils
from tensorflow.keras.applications import VGG16, MobileNetV2, ResNet50, DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

In [ ]:
class FaceDetectionSystem:
    def __init__(self, base_path, img_size=(224, 224)):
        self.base_path = base_path
        self.img_size = img_size
        self.categories = ['WithMask', 'WithoutMask']
        self.models = {}
        self.histories = {}
        self.results = {}

        # Initialize face detector for multi-face detection
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    def load_and_preprocess_data(self):
        """Load and preprocess data with face detection"""
        print("Loading and preprocessing data...")
        data = []

        for label, category in enumerate(self.categories):
            cat_dir = os.path.join(self.base_path, category)
            if not os.path.exists(cat_dir):
                print(f"Warning: Directory {cat_dir} does not exist")
                continue

            for fname in os.listdir(cat_dir):
                img_path = os.path.join(cat_dir, fname)
                if os.path.isfile(img_path) and fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img = cv2.imread(img_path)
                    if img is not None:
                        # Detect faces in the image
                        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                        faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)

                        if len(faces) > 0:
                            # Process each detected face
                            for (x, y, w, h) in faces:
                                face_img = img[y:y+h, x:x+w]
                                face_img = cv2.resize(face_img, self.img_size)
                                data.append((face_img, label))
                        else:
                            # If no face detected, use whole image
                            img = cv2.resize(img, self.img_size)
                            data.append((img, label))

        random.shuffle(data)
        X = np.array([d[0] for d in data])
        y = np.array([d[1] for d in data])

        print(f"Loaded {len(X)} images")
        print(f"Class distribution: {np.bincount(y)}")

        return X, y

    def prepare_data(self, X, y):
        """Split and prepare data for training"""
        X_train, X_temp, y_train, y_temp = train_test_split(
            X, y, test_size=0.3, stratify=y, random_state=42
        )
        X_val, X_test, y_val, y_test = train_test_split(
            X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
        )

        # Convert to categorical
        y_train_cat = utils.to_categorical(y_train, num_classes=2)
        y_val_cat = utils.to_categorical(y_val, num_classes=2)
        y_test_cat = utils.to_categorical(y_test, num_classes=2)

        return (X_train, X_val, X_test), (y_train_cat, y_val_cat, y_test_cat), (y_train, y_val, y_test)

    def create_custom_model(self):
        """Create custom CNN model from scratch"""
        model = models.Sequential([
            layers.Input(shape=(224, 224, 3)),
            layers.Rescaling(1.0 / 255),

            # First Conv Block
            layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
            layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.25),

            # Second Conv Block
            layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
            layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.25),

            # Third Conv Block
            layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
            layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.25),

            # Fourth Conv Block
            layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
            layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.25),

            # Classification Head
            layers.GlobalAveragePooling2D(),
            layers.Dense(512, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(2, activation='softmax')
        ])

        return model

    def create_transfer_learning_model(self, base_model_name):
        """Create transfer learning model"""
        if base_model_name == 'VGG16':
            base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
        elif base_model_name == 'MobileNetV2':
            base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
        elif base_model_name == 'ResNet50':
            base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
        elif base_model_name == 'DenseNet121':
            base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

        base_model.trainable = False

        model = models.Sequential([
            layers.Input(shape=(224, 224, 3)),
            layers.Rescaling(1.0 / 255),
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(2, activation='softmax')
        ])

        return model

    def train_model(self, model, model_name, X_train, X_val, y_train, y_val):
        """Train a model with proper callbacks"""
        print(f"\nTraining {model_name}...")

        # Compile model
        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        # Data augmentation
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
        )
        datagen.fit(X_train)

        # Callbacks
        early_stop = callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            verbose=1,
            restore_best_weights=True
        )

        reduce_lr = callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=3,
            min_lr=0.0001,
            verbose=1
        )

        checkpoint = callbacks.ModelCheckpoint(
            f'best_{model_name.lower()}_model.h5',
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        )

        # Training
        history = model.fit(
            datagen.flow(X_train, y_train, batch_size=32),
            steps_per_epoch=len(X_train) // 32,
            epochs=25,
            validation_data=(X_val, y_val),
            callbacks=[early_stop, reduce_lr, checkpoint],
            verbose=1
        )

        return model, history

    def evaluate_model(self, model, model_name, X_test, y_test, y_test_orig):
        """Evaluate model and return metrics"""
        print(f"\nEvaluating {model_name}...")

        # Predictions
        y_pred = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)

        # Metrics
        test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

        # Classification report
        report = classification_report(
            y_test_orig,
            y_pred_classes,
            target_names=self.categories,
            output_dict=True
        )

        # Confusion matrix
        cm = confusion_matrix(y_test_orig, y_pred_classes)

        return {
            'test_loss': test_loss,
            'test_accuracy': test_accuracy,
            'classification_report': report,
            'confusion_matrix': cm,
            'predictions': y_pred_classes,
            'true_labels': y_test_orig
        }

    def plot_improved_confusion_matrix(self, cm, model_name):
        """Plot improved confusion matrix"""
        plt.figure(figsize=(10, 8))

        # Create annotation matrix
        group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
        group_counts = [f"{value:0.0f}" for value in cm.flatten()]
        group_percentages = [f"{value:.1%}" for value in cm.flatten()/np.sum(cm)]

        labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names, group_counts, group_percentages)]
        labels = np.asarray(labels).reshape(2, 2)

        # Create heatmap
        sns.heatmap(
            cm,
            annot=labels,
            fmt='',
            cmap='Blues',
            cbar_kws={'label': 'Count'},
            xticklabels=self.categories,
            yticklabels=self.categories,
            linewidths=2,
            linecolor='white'
        )

        plt.title(f'{model_name} - Confusion Matrix\n', fontsize=16, fontweight='bold')
        plt.xlabel('Predicted Label', fontsize=12)
        plt.ylabel('True Label', fontsize=12)
        plt.tight_layout()
        plt.savefig(f'confusion_matrix_{model_name.lower()}.png', dpi=300, bbox_inches='tight')
        plt.show()

    def plot_training_history(self, history, model_name):
        """Plot training history with val_accuracy vs val_loss"""
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))

        # Accuracy plot
        axes[0, 0].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
        axes[0, 0].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
        axes[0, 0].set_title(f'{model_name} - Model Accuracy')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Accuracy')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)

        # Loss plot
        axes[0, 1].plot(history.history['loss'], label='Train Loss', linewidth=2)
        axes[0, 1].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
        axes[0, 1].set_title(f'{model_name} - Model Loss')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('Loss')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)

        # Val Accuracy vs Val Loss (Teacher's specific requirement)
        axes[1, 0].plot(history.history['val_loss'], history.history['val_accuracy'],
                       'bo-', linewidth=2, markersize=6)
        axes[1, 0].set_title(f'{model_name} - Val Accuracy vs Val Loss')
        axes[1, 0].set_xlabel('Validation Loss')
        axes[1, 0].set_ylabel('Validation Accuracy')
        axes[1, 0].grid(True, alpha=0.3)

        # Learning rate (if available)
        if 'lr' in history.history:
            axes[1, 1].plot(history.history['lr'], linewidth=2)
            axes[1, 1].set_title(f'{model_name} - Learning Rate')
            axes[1, 1].set_xlabel('Epoch')
            axes[1, 1].set_ylabel('Learning Rate')
            axes[1, 1].set_yscale('log')
            axes[1, 1].grid(True, alpha=0.3)
        else:
            axes[1, 1].text(0.5, 0.5, 'Learning Rate\nNot Recorded',
                          ha='center', va='center', transform=axes[1, 1].transAxes)

        plt.tight_layout()
        plt.savefig(f'training_history_{model_name.lower()}.png', dpi=300, bbox_inches='tight')
        plt.show()

        # Explain why val_loss might increase
        print(f"\n--- Analysis for {model_name} ---")
        print("Why validation loss might increase:")
        print("1. Overfitting: Model memorizes training data but fails on validation")
        print("2. Learning rate too high: Causes oscillations in loss")
        print("3. Insufficient regularization: Model complexity exceeds data complexity")
        print("4. Data imbalance: Model biased towards majority class")
        print("5. Batch size effects: Small batches can cause noisy gradients")

    def detect_multiple_faces(self, image_path, model):
        """Detect and classify multiple faces in an image"""
        img = cv2.imread(image_path)
        if img is None:
            return None, None

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)

        results = []
        img_with_boxes = img.copy()

        for (x, y, w, h) in faces:
            # Extract face
            face_img = img[y:y+h, x:x+w]
            face_resized = cv2.resize(face_img, self.img_size)
            face_normalized = face_resized.astype('float32') / 255.0
            face_batch = np.expand_dims(face_normalized, axis=0)

            # Predict
            prediction = model.predict(face_batch, verbose=0)
            class_idx = np.argmax(prediction)
            confidence = np.max(prediction)
            label = self.categories[class_idx]

            results.append({
                'bbox': (x, y, w, h),
                'label': label,
                'confidence': confidence
            })

            # Draw bounding box
            color = (0, 255, 0) if label == 'with_mask' else (0, 0, 255)
            cv2.rectangle(img_with_boxes, (x, y), (x+w, y+h), color, 2)
            cv2.putText(img_with_boxes, f'{label}: {confidence:.2f}',
                       (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        return results, img_with_boxes

    def run_complete_pipeline(self):
        """Run the complete face detection pipeline"""
        print("=== Face Detection System - Complete Pipeline ===")

        # Load data
        X, y = self.load_and_preprocess_data()
        (X_train, X_val, X_test), (y_train, y_val, y_test), (y_train_orig, y_val_orig, y_test_orig) = self.prepare_data(X, y)

        # Model names to train
        model_names = ['Custom_CNN', 'VGG16', 'MobileNetV2', 'ResNet50', 'DenseNet121']

        # Train all models
        for model_name in model_names:
            print(f"\n{'='*50}")
            print(f"Processing {model_name}")
            print(f"{'='*50}")

            # Create model
            if model_name == 'Custom_CNN':
                model = self.create_custom_model()
            else:
                model = self.create_transfer_learning_model(model_name)

            print(f"{model_name} Model Summary:")
            model.summary()

            # Train model
            trained_model, history = self.train_model(
                model, model_name, X_train, X_val, y_train, y_val
            )

            # Store results
            self.models[model_name] = trained_model
            self.histories[model_name] = history

            # Evaluate model
            results = self.evaluate_model(
                trained_model, model_name, X_test, y_test, y_test_orig
            )
            self.results[model_name] = results

            # Plot results
            self.plot_training_history(history, model_name)
            self.plot_improved_confusion_matrix(results['confusion_matrix'], model_name)

            # Print results
            print(f"\n{model_name} Results:")
            print(f"Test Accuracy: {results['test_accuracy']:.4f}")
            print(f"Test Loss: {results['test_loss']:.4f}")
            print("\nClassification Report:")
            for class_name in self.categories:
                metrics = results['classification_report'][class_name]
                print(f"{class_name}: Precision={metrics['precision']:.3f}, "
                      f"Recall={metrics['recall']:.3f}, F1={metrics['f1-score']:.3f}")

        # Compare all models
        self.compare_models()

        return self.models, self.results

    def compare_models(self):
        """Compare all models performance"""
        print("\n" + "="*60)
        print("MODEL COMPARISON SUMMARY")
        print("="*60)

        comparison_data = []
        for model_name, results in self.results.items():
            comparison_data.append({
                'Model': model_name,
                'Accuracy': results['test_accuracy'],
                'Loss': results['test_loss'],
                'With_Mask_F1': results['classification_report']['with_mask']['f1-score'],
                'Without_Mask_F1': results['classification_report']['without_mask']['f1-score']
            })

        # Sort by accuracy
        comparison_data.sort(key=lambda x: x['Accuracy'], reverse=True)

        print(f"{'Model':<15} {'Accuracy':<10} {'Loss':<10} {'With_Mask_F1':<12} {'Without_Mask_F1':<15}")
        print("-" * 70)
        for data in comparison_data:
            print(f"{data['Model']:<15} {data['Accuracy']:<10.4f} {data['Loss']:<10.4f} "
                  f"{data['With_Mask_F1']:<12.4f} {data['Without_Mask_F1']:<15.4f}")

        # Plot comparison
        plt.figure(figsize=(12, 6))
        models = [d['Model'] for d in comparison_data]
        accuracies = [d['Accuracy'] for d in comparison_data]

        bars = plt.bar(models, accuracies, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'])
        plt.title('Model Performance Comparison', fontsize=16, fontweight='bold')
        plt.xlabel('Models')
        plt.ylabel('Test Accuracy')
        plt.xticks(rotation=45)

        # Add value labels on bars
        for bar, acc in zip(bars, accuracies):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                    f'{acc:.3f}', ha='center', va='bottom')

        plt.tight_layout()
        plt.grid(True, alpha=0.3)
        plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
        plt.show()

In [ ]:
# Example usage and application
def create_face_detection_app():
    """Create a simple face detection application"""
    print("=== Face Detection Application ===")

    # Initialize the system
    # Note: Update this path to your actual dataset path
    #from google.colab import drive
   # base_path = drive.mount('/content/drive')  # Update this path
    base_path = "/content/drive/MyDrive/Face Mask Dataset/Train"
    system = FaceDetectionSystem(base_path)

    # Check if models exist, if not train them
    if not os.path.exists('best_custom_cnn_model.h5'):
        print("Training models...")
        models, results = system.run_complete_pipeline()

        # Save comparison results
        with open('model_comparison_results.json', 'w') as f:
            json.dump({
                name: {
                    'accuracy': float(results['test_accuracy']),
                    'loss': float(results['test_loss'])
                } for name, results in results.items()
            }, f, indent=2)
    else:
        print("Loading existing models...")
        # Load best model (you can modify this to load your preferred model)
        system.models['Custom_CNN'] = models.load_model('best_custom_cnn_model.h5')

    return system

In [ ]:
# Main execution
if __name__ == "__main__":
    # Create the face detection system
    system = create_face_detection_app()

    # If you want to test on a specific image with multiple faces
    # Uncomment and modify the path below:
    """
    test_image_path = "/content/drive/MyDrive/Face-Detection/train/0-with-mask_jpg.rf.2dd114e4f143ba8bf221a0377529b7a5.jpg"
    if os.path.exists(test_image_path):
        results, img_with_boxes = system.detect_multiple_faces(
            test_image_path,
            system.models['Custom_CNN']
        )

        if results:
            print(f"Detected {len(results)} faces:")
            for i, result in enumerate(results):
                print(f"Face {i+1}: {result['label']} (confidence: {result['confidence']:.3f})")

            # Display result
            plt.figure(figsize=(12, 8))
            plt.imshow(cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB))
            plt.title('Multi-Face Detection Results')
            plt.axis('off')
            plt.show()
    """

    print("\n=== Setup Complete ===")
    print("To use the system:")
    print("1. Update the base_path variable to point to your dataset")
    print("2. Ensure your dataset has 'with_mask' and 'without_mask' folders")
    print("3. Run the complete pipeline to train and compare all models")
    print("4. Use detect_multiple_faces() for testing on new images")

=== Face Detection Application ===
Training models...
=== Face Detection System - Complete Pipeline ===
Loading and preprocessing data...
Loaded 10148 images
Class distribution: [5113 5035]

Processing Custom_CNN
Custom_CNN Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,435,682 (5.48 MB)

 Trainable params: 1,435,682 (5.48 MB)

 Non-trainable params: 0 (0.00 B)


Training Custom_CNN...
